# Exercicio 2:   


    Um sistema de tráfego  é representado por um grafo orientado ligado. Os nodos denotam pontos de acesso e  os arcos denotam vias de comunicação só com um sentido .  O grafo tem de ser ligado: entre cada par de nodos e <n1,n2> tem de existir um caminho n1 -> n2 e um caminho n2 -> n1. \
 

# Bibliotecas a utilizar:
## 1) networkx:
    - Biblioteca que ajuda na criação e manipulação de grafos. Com uma interface simples para trabalhar com grafos orientados e não orientados;
    - Neste trablho vamos trabalhar com grafos orientados e verificar se os grafos são orientados e ligados. E por isso, com a utilização desta biblioteca vamos conseguir manipular arestas e nós do grafo

## 2) ortools: 
    - A biblioteca ortools é uma ferramenta que permite resolver problemas de programação linear, inteira e problemas de otimização com restrições utilizando diferentes tipos de **solvers**;
    - Vamos utilizar esta biblioteca neste trabalho com o propósito no problema de remoção de arestas, para garantir que um grafo é conexo. O **ortools** vai maximizar o número de arestas que poderão ser removidas enquanto mantém as restrições.

## 3) random:
    - A biblioteca "**random**" é um biblioteca de Python que fornece funções de gerações de números aleatórios e apresentações aleatórias;
    - Vamos utilizar esta bilbioteca para gerar descendentes aleatórios de nós no grafo, ou seja, na criação de arestas entre os nós é feita de maneira aleatória com base nos limites definidos;  

In [1]:
import networkx as nx
from ortools.linear_solver import pywraplp
import random 

### Alínea a)
    - Vamos ter que gerar um grafo orientado aleatório com N(número máximo de nodos) entre 6 a 10, onde cada nodo tem entre 1 a 3 descendentes, sem loops ou destinos que se repitam;
    - O grafo tem que ser orientado ligado, ou seja, cada para **<n1,n2>**, deve haver um caminho de n1 para n2 e outro caminho de n2 para n1.

In [2]:
### Completar
N = random.randint(6,10)

def geraGrafo(N): 
    while True: 
        G = nx.DiGraph()
        G.add_nodes_from(range(N))

        for node in range(N):
            desc = random.randint(1,3)
            dest = random.sample([n for n in range(N) if n != node], desc)
            G.add_edges_from((node, dest) for dest in dest)

        if nx.is_strongly_connected(G):
            return G

grafo = geraGrafo(N)



    Neste bloco de codigo definimos a variável N sendo o número máximo de nodos entre 6 a 10 gerados aleatóriamente com o auxilio da biblioteca *random*. Depois contruimos uma função para gerar   grafos de acordo com a varável N definida anteriormente.
    A condição <i>while True:</i> permite que os grafos gerados sejam sempre orientados ligados, se não for utilizado esta condição teriamos que ir por uma abordagem mais "controlada", ou seja, teriamos que definir um número máximo de tentativas para gerar um grafo orientado ligado para evitar um loop infinito caso a geração falhe várias vezes. A função *geraGrafo* vai criar um grafo G primeiramente vazio e de seguida adiciona o número de nodes dado aleatóriamente por N. O *for* seguinte vai é um ciclo que vai adicionar arestas de acordo com as restrições dos descendentes e o <i>if statement</i> utiliza a função <bold>is_strongly_connected</bold> que verifica se o grafo é conexo ou não, se sim vai gerar o mesmo caso contrário não.
    

In [3]:
for w,r in grafo.edges():
    print(f"{w} -> {r}")

0 -> 2
1 -> 4
1 -> 2
2 -> 7
2 -> 0
2 -> 1
3 -> 6
3 -> 5
4 -> 0
5 -> 1
5 -> 4
5 -> 0
6 -> 5
6 -> 2
7 -> 4
7 -> 0
7 -> 3


Este bloco de código é apenas um loop para gerar as arestas do grafo

In [5]:
# Código Inteiro:
N = random.randint(6,10)

def geraGrafo(N): 
    while True: 
        G = nx.DiGraph()
        G.add_nodes_from(range(N))

        for node in range(N):
            desc = random.randint(1,3)
            dest = random.sample([n for n in range(N) if n != node], desc)
            G.add_edges_from((node, dest) for dest in dest)

        if nx.is_strongly_connected(G):
            return G

grafo = geraGrafo(N)

for w,r in grafo.edges():
    print(f"{w} -> {r}")


0 -> 4
1 -> 5
1 -> 2
1 -> 3
2 -> 1
2 -> 4
3 -> 0
3 -> 1
4 -> 1
4 -> 0
5 -> 0
